In [1]:
import pandas as pd
import numpy as np
import time
from numba import vectorize
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('/home/ak/research/emmanuel/proj_algorithm/NormalCode/sim_pulse_02_updated.txt', sep = '    ')
time_val = np.array(df['time_sec'])
sample_time = df['time_sec'][1] - df['time_sec'][0] 
lower_limit_freq = 2/(max(df['time_sec']))
upper_limit_freq = 1/(2*sample_time)
df.head(1)

,time_sec,intensity
0,0.0,0.77546


In [3]:
@vectorize(['float32(float32, float32, float32)'], target = 'cuda')
def pulse_phase_numba(p0, t, w0):    
    """
    Calculates and returns pulse phase at any given time 't' when acceleration 'a', initial 
    phase 'p0' and angular frequency 'w0' are known.
    """
    phase_values = p0 + 2.0*np.pi*w0*t + 0.5**2*t**2
    return phase_values

CudaSupportError: Error at driver init: 

CUDA driver library cannot be found.
If you are sure that a CUDA driver is installed,
try setting environment variable NUMBA_CUDA_DRIVER
with the file path of the CUDA driver shared library.
:

In [7]:
N_phase = [4, 8, 16, 32]
N_freq =  [8, 16, 32, 64]
total_num_of_computation = []
total_time_of_computation = []

    
for k1 in range(len(N_freq)):
    phase_stamps = np.repeat(np.linspace(0,2.0*np.pi, N_phase[k1]), 146454*N_freq[k1]).astype('float32')
    freq_values = np.repeat(np.linspace(lower_limit_freq, upper_limit_freq, N_freq[k1]), 146454).astype('float32')
    freq_stamps = np.tile(freq_values, N_phase[k1])
    time_stamps = np.tile(time_val, N_phase[k1]*N_freq[k1]).astype('float32')
    
    
    clock_start = time.time()
    calculated_values = pulse_phase_numba(phase_stamps, time_stamps, freq_stamps)
    clock_end = time.time()
    
    time_taken = clock_end - clock_start
    num_of_computation = N_phase[k1]*N_freq[k1]*146454
    total_num_of_computation.append(num_of_computation)
    total_time_of_computation.append(time_taken)
    if k1 == 3:
        print(calculated_values)

NameError: name 'pulse_phase_numba' is not defined

In [9]:
print('\nTotal number of calculation in each cycle are: ', total_num_of_computation)
print('\nTotal time taken for calculation are: ', total_time_of_computation)


Total number of calculation in each cycle are:  []

Total time taken for calculation are:  []
